## 벡터화 실습: 원-핫 인코딩 구현해보기
한국어로 실습하기 위해서 한국어 형태소 분석기 패키지 KoNLPy를 설치합니다.

In [1]:
# pip install konlpy

In [2]:
import re
from konlpy.tag import Okt
from collections import Counter

In [3]:
text = "임금님 귀는 당나귀 귀! 임금님 귀는 당나귀 귀! 실컷~ 소리치고 나니 속이 확 뚫려 살 것 같았어."
text

'임금님 귀는 당나귀 귀! 임금님 귀는 당나귀 귀! 실컷~ 소리치고 나니 속이 확 뚫려 살 것 같았어.'

### 전처리
이 텍스트에는' !, ~, . '와 같은 각종 특수문자들이 있습니다. 상황에 따라 다르겠지만, 대개 이런 특수문자들은 자연어 처리에서 큰 의미를 가지지 못합니다. 여기서는 정규 표현식을 사용하여 특수문자들을 제거하고자 합니다.  
한글과 공백을 제외하고 특수문자만 제거하고 싶다면 정규 표현식으로 한국어의 범위를 지정할 수 있어야 합니다. 일반적으로 자음의 범위는 'ㄱ ~ ㅎ', 모음의 범위는 'ㅏ ~ ㅣ;와 같이 지정할 수 있습니다. 

In [4]:
reg = re.compile("[^ㄱ-ㅎㅏ-ㅣ가-힣]")
text = reg.sub(' ',text)
print(text)

임금님 귀는 당나귀 귀  임금님 귀는 당나귀 귀  실컷  소리치고 나니 속이 확 뚫려 살 것 같았어 


### 토큰화

단어장을 구성하기 위해서는 단어장의 원소인 토큰(token)이라는 단위를 정해줄 필요가 있습니다. 그리고 한국어는 주로 형태소 분석기를 통해서 토큰 단위를 나눠줍니다. 여기서는 KoNLPy에 내장된 Okt 형태소 분석기를 사용해보겠습니다.

In [5]:
okt=Okt()
tokens = okt.morphs(text)
print(tokens)

['임금님', '귀', '는', '당나귀', '귀', '임금님', '귀', '는', '당나귀', '귀', '실컷', '소리', '치고', '나니', '속이', '확', '뚫려', '살', '것', '같았어']


### 단어장 만들기

토큰들을 가지고 단어장을 만들어 볼 텐데요. 여기서는 빈도수가 높은 단어일수록 낮은 정수를 부여하려고 합니다. 빈도수가 높은 순서대로 낮은 정수를 부여하기 위해서 각 단어의 빈도수를 카운트 해주어야 합니다. 따라서 파이썬의 Counter 서브클래스를 사용해서 단어의 빈도를 카운트 하겠습니다.

In [6]:
vocab = Counter(tokens)
print(vocab)

Counter({'귀': 4, '임금님': 2, '는': 2, '당나귀': 2, '실컷': 1, '소리': 1, '치고': 1, '나니': 1, '속이': 1, '확': 1, '뚫려': 1, '살': 1, '것': 1, '같았어': 1})


In [7]:
vocab['임금님']

2

In [8]:
vocab_size = 5
vocab = vocab.most_common(vocab_size) # 등장 빈도수가 높은 상위 5개의 단어만 저장
print(vocab)

[('귀', 4), ('임금님', 2), ('는', 2), ('당나귀', 2), ('실컷', 1)]


In [9]:
word2idx={word[0] : index+1 for index, word in enumerate(vocab)}
print(word2idx)

{'귀': 1, '임금님': 2, '는': 3, '당나귀': 4, '실컷': 5}


### 원-핫 벡터 만들기
원-핫 인코딩을 하는 함수를 만들어 각 단어를 원-핫 벡터로 만들겠습니다. 아래의 함수는 특정 단어와 단어장을 입력하면 해당 단어의 원-핫 벡터를 리턴하는 함수입니다.

In [10]:
def one_hot_encoding(word, word2index):
        one_hot_vector = [0]*(len(word2index))
        index = word2index[word]
        one_hot_vector[index-1] = 1
        return one_hot_vector

In [11]:
one_hot_encoding("임금님", word2idx)

[0, 1, 0, 0, 0]

### 케라스를 통한 원-핫 인코딩(one-hot encoding)
텐서플로의 케라스 API를 사용해 원핫인코딩을 진행하겠습니다.

In [12]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.utils import to_categorical

In [13]:
text = [['강아지', '고양이', '강아지'],['애교', '고양이'], ['컴퓨터', '노트북']]
text

[['강아지', '고양이', '강아지'], ['애교', '고양이'], ['컴퓨터', '노트북']]

In [14]:
t = Tokenizer()
t.fit_on_texts(text)
print(t.word_index) # 각 단어에 대한 인코딩 결과 출력

{'강아지': 1, '고양이': 2, '애교': 3, '컴퓨터': 4, '노트북': 5}


 이제 단어장의 크기를 vocab_size라는 변수에 저장해두겠습니다.

In [15]:
vocab_size = len(t.word_index) + 1

In [16]:
len(t.word_index)

5

vocab_size에 1을 더해주는 이유는 케라스 토크나이저는 각 단어에 고유한 정수를 부여할 때, 숫자 1부터 부여하지만 실제로 자연어 처리를 할 때는 특별 토큰으로 0번 단어로 단어장에 추가로 사용하는 경우가 많기 때문입니다. 주로 0번은 패딩(padding) 작업을 위한 패딩 토큰으로 사용됩니다.  
케라스 토크나이저에 단어장이 저장되었으므로, 단어장에 속한 단어들로 구성된 텍스트 시퀀스는 케라스 토크나이저를 통해 정수 시퀀스로 변환할 수 있습니다.

In [17]:
sub_text = ['강아지', '고양이', '강아지', '컴퓨터']
encoded = t.texts_to_sequences([sub_text])
print(encoded)

[[1, 2, 1, 4]]


텍스트 시퀀스가 정수 시퀀스로 변환되는 것을 보여줍니다. 앞에서 강아지는 1번, 고양이는 2번, 컴퓨터는 4번으로 정수가 부여되었습니다. 이렇게 변환된 정수 시퀀스는 to_categorical()을 사용해 원-핫 벡터의 시퀀스로 변환할 수 있습니다.

In [18]:
one_hot = to_categorical(encoded, num_classes=vocab_size)
print(one_hot)

[[[0. 1. 0. 0. 0. 0.]
  [0. 0. 1. 0. 0. 0.]
  [0. 1. 0. 0. 0. 0.]
  [0. 0. 0. 0. 1. 0.]]]


## 워드 임베딩

### 희소 벡터의 문제점
희소 벡터에는 차원의 저주(curse of dimensionality)라는 문제가 있습니다. 또한 유사도를 구할 수 없기도 합니다.  
벡터 간 유사도를 구하는 방법으로는 대표적으로 내적(inner product)이 있습니다. 임의의 두개의 원-핫 벡터 간 내적(inner product)을 구해보면, 서로 직교(orthogonal)하여 그 값은 0입니다. 이는 모든 원-핫 벡터의 상호 유사도가 0임을 의미하며. 결국 원-핫 벡터를 통해서는 단어 벡터 간 유사도를 구할 수 없음을 의미합니다.
위와 같은 문제점을 해결하기 위해서 '기계가 단어장 크기보다 적은 차원의 밀집 벡터(dense vector)를 학습'하는 **워드 임베딩(word embedding)**이 제안되었습니다. 이를 통해 얻는 밀집 벡터는 각 차원이 0과 1이 아닌 다양한 실숫값을 가지며, 이 밀집 벡터를 임베딩 벡터(embedding vector)라고 합니다.

### 워드 임베딩(Word Embedding)
밀집 표현은 벡터의 차원을 단어 집합의 크기로 상정하지 않습니다. 사용자가 설정한 값으로 모든 단어의 벡터 표현의 차원을 맞춥니다.  

## Word2Vec1. 분포가설
Word2Vec의 핵심 아이디어는 분포 가설(distributional hypothesis)을 따릅니다.  
분포 가설 : ‘비슷한 문맥에서 같이 등장하는 경향이 있는 단어들은 비슷한 의미를 가진다.’  
분포 가설에 따르는 Word2Vec은 같이 등장하는 경향이 적은 단어들에 비해 '강아지', '애교, '귀여운'과 같은 단어들을 상대적으로 유사도가 높은 벡터로 만듭니다.

## Word2Vec2. CBoW
Word2Vec에는 크게 CBoW와 `Skip-gram'라는 두 가지 방법이 있습니다. CBoW는 주변에 있는 단어들을 통해 중간에 있는 단어들을 예측하는 방법입니다. 반대로, Skip-Gram은 중간에 있는 단어로 주변 단어들을 예측하는 방법입니다. 메커니즘 자체는 거의 동일합니다.

### CBoW(Continuous Bag of words)
예문 : "I like natural language processing."  
위와 같은 문장이 있다고 합시다. CBoW는 중간에 있는 단어를 예측하는 방법이므로 {"i", "like", "language", "processing"}으로부터 "natural"을 예측하는 것은 CBoW가 하는 일입니다. 이때 예측해야 하는 단어 "natural"을 중심 단어(center word)라고 하고, 예측에 사용되는 단어들을 주변 단어(context word)라고 합니다.  
중심 단어를 예측하기 위해 앞, 뒤로 몇 개의 단어를 볼지를 결정를 결정했다면, 그 범위를 윈도우(window)라고 합니다. 만약 윈도우 크기가 1이고, 예측하고자 하는 중심 단어가 "language"라면 앞의 한 단어인 "natural"과 뒤의 한 단어인 "processing"을 참고합니다. 윈도우 크기가 m일 때, 중심 단어를 예측하기 위해 참고하는 주변 단어의 개수는 2m입니다.  
윈도우 크기를 정했다면, 윈도우를 계속 움직여서 주변 단어와 중심 단어를 바꿔가며 학습을 위한 데이터 셋을 만들 수 있는데, 이 방법을 **슬라이딩 윈도우(sliding window)**  
* 아래 데이터셋의 형식은 ((주변 단어의 셋), 중심 단어)임을 가정한다.
* ((like), I), ((I, natural), like), ((like, language), natural), ((natural, processing), language), ((language), processing)  

이렇게 선택된 데이터셋에서 단어 각각은 원-핫 인코딩되어 원-핫 벡터가 되고, 원-핫 벡터가 CBoW나 Skip-gram의 입력이 됩니다. 윈도우 크기가 m이라면 2m개의 주변 단어를 이용해 1개의 중심 단어를 예측하는 과정에서 두 개의 가중치 행렬(matrix)을 학습하는 것이 목적입니다.  
Word2Vec은 은닉층이 1개라서 딥 러닝이라기보다는 얕은 신경망(Shallow Neural Network)을 학습한다고 볼 수 있습니다.


CBoW 신경망 구조에서 주변 단어 각각의 원-핫 벡터는 입력층에 위치하고 중심 단어의 원-핫 벡터가 위치한 곳은 출력층이라고 볼 수 있습니다. CBoW에서 사실 입력층과 출력층의 크기는 단어 집합의 크기인 VV로 이미 고정되어 있습니다. 하지만 은닉층의 크기는 사용자가 정의해주는 하이퍼파라미터입니다.  
주변 단어로 선택된 각각의 원-핫 벡터는 첫 번째 가중치 행렬과 곱해지게 됩니다. 이때 가중치 행렬의 크기는 (V × N)입니다. 그런데 원-핫 벡터는 각 단어의 정수 인덱스 i에 해당되는 위치에만 1의 값을 가지므로, 원-핫 벡터와 가중치 행렬과의 곱은 가중치 행렬의 i 위치에 있는 행을 그대로 가져오는 것과 동일합니다.  
이를 마치 테이블에서 값을 그대로 룩업(lookup)해오는 것과 같다고 하여 룩업 테이블(lookup table)이라고 합니다.

룩업 테이블을 거쳐서 생긴 2m개의 주변 단어 벡터들은 각각 N의 크기를 가집니다. CBoW에서는 이 벡터들을 모두 합하거나, 평균을 구한 값을 최종 은닉층의 결과로 합니다. 그러면 최종 은닉층의 결과도 N차원의 벡터가 되겠죠. 이게 은닉층 연산의 전부입니다. Word2Vec에서는 은닉층에서 활성화 함수나 편향(bias)을 더하는 연산을 하지 않습니다.  
Word2Vec에서의 은닉층은 활성화 함수가 존재하지 않고, 단순히 가중치 행렬과의 곱셈만을 수행하기에 기존 신경망의 은닉층과 구분 지어 투사층(projection layer)이라고도 합니다.

은닉층에서 생성된 N차원의 벡터는 두 번째 가중치 행렬과 곱해집니다. 이 가중치 행렬의 크기는 (NN × VV)이므로, 곱셈의 결과로 나오는 벡터의 차원은 V입니다. 출력층은 활성화 함수로 소프트맥스 함수를 사용하므로 이 V차원의 벡터는 활성화 함수를 거쳐 모든 차원의 총합이 1이 되는 벡터로 변경됩니다.  
CBoW는 이 출력층의 벡터를 중심 단어의 원-핫 벡터와의 손실(loss)을 최소화하도록 학습시킵니다. 이 과정에서 첫 번째 가중치 행렬 W와 두 번째 가중치 행렬 W'가 업데이트되는데 학습이 다 되었다면 N차원의 크기를 갖는 W의 행이나 W'의 열로부터 어떤 것을 임베딩 벡터로 사용할지를 결정하면 됩니다. 때로는 W와 W'의 평균치를 임베딩 벡터로 선택하기도 합니다.

## Word2Vec3. Skip-gram과 Negative Sampling
### Skip-gram
Skip-gram은 CBoW와는 달리 중심 단어로부터 주변 단어 각각을 예측합니다. 이런 특성으로 인해 데이터셋 또한 다릅니다.  
* 아래 데이터셋의 형식은 (중심 단어, 주변 단어)임을 가정합니다.
* (i, like) (like, I), (like, natural), (natural, like), (natural, language), (language, natural), (language, processing), (processing, language)  
중심 단어로부터 주변 단어를 예측한다는 점, 그리고 이로 인해 중간에 은닉층에서 다수의 벡터의 덧셈과 평균을 구하는 과정이 없어졌다는 점만 제외하면 CBoW와 메커니즘 자체는 동일합니다. skip-gram도 CBoW와 마찬가지로 학습 후에 가중치 행렬 W의 행 또는 W'의 열로부터 임베딩 벡터를 얻을 수 있습니다

### 네거티브 샘플링(negative sampling)
대체적으로 Word2Vec를 사용할 때는 SGNS(Skip-Gram with Negative Sampling)을 사용합니다. 즉 Skip-gram을 사용하면서 네거티브 샘플링(Negative Sampling)이란 방법도 사용한다는 것입니다. Word2Vec의 구조는 연산량이 지나치게 많아 실제로 사용하기 어렵기 때문입니다.  
Skip-gram의 학습 과정을 살펴보면  모델 구조는 단순해 보이지만 복잡한 과정을 거칩니다. 출력층에서 소프트맥스 함수를 통과한 V 차원의 벡터와 레이블에 해당되는 V차원의 주변 단어의 원-핫 벡터와의 오차를 구하고, 역전파를 통해 모든 단어에 대한 임베딩 벡터을 조정합니다. 그 단어가 중심 단어나 주변 단어와 전혀 상관없는 단어라도요. 만약 단어장의 크기가 수십, 수백만에 달한다면 이 작업은 너무너무 느립니다! Output을 위한 소프트맥스(Softmax) 함수의 분모항이 수백만에 달하는 것만 생각해 보아도 충분히 예상할 수 있습니다.  
하지만 지금 집중하고 있는 중심 단어와 주변 단어가 '사과', '딸기'와 같이 과일과 관련된 단어라면, '필통', '연필'이라는 연관 관계가 없는 단어들의 임베딩 값을 굳이 업데이트할 필요가 없습니다. 네거티브 샘플링은 연산량을 줄이기 위해서 소프트맥스 함수를 사용한 V개 중 1개를 고르는 다중 클래스 분류 문제를 시그모이드 함수를 사용한 이진 분류 문제로 바꾸기로 합니다.

네거티브 샘플링은 중심 단어와 주변 단어를 입력값으로 받아 이 두 단어가 정말로 이웃 관계면(실제로 중심 단어와 주변 단어의 관계면) 1을 또는 0을 출력하는 문제로 바꿉니다. 기존의 다중 분류 문제에서 이진 분류 문제로 바뀐 것입니다.  
이는 랜덤으로 단어장에 있는 아무 단어나 가져와 target word로 하는 거짓 데이터셋을 만들고 0으로 레이블링을 해주는 것입니다. 거짓(negative) 데이터셋을 만들기 때문에 이 방법이 네거티브 샘플링이라 불립니다.  
이렇게 완성된 데이터셋으로 학습하면 Word2Vec은 더 이상 다중 클래스 분류 문제가 아니라 이진 분류 문제로 간주할 수 있습니다. 중심 단어와 주변 단어를 내적하고, 출력층의 시그모이드 함수를 지나게 하여 1 또는 0의 레이블로부터 오차를 구해서 역전파를 수행합니다.  
이런 학습 방식은 기존의 소프트맥스 함수를 사용했던 방식보다 상당량의 연산량을 줄일 수 있는 효과를 가지고 있습니다.  이 아이디어는 엄청난 연산량을 필요로 했던 Word2Vec의 학습이 가능케 했던 핵심적인 아이디어 중 하나입니다.

## Word2Vec4. 영어 Word2Vec 실습과 OOV 문제
사용할 훈련 데이터는 NLTK에서 제공하는 코퍼스이며, gensim 패키지는 토픽 모델링을 위한 NLP 패키지입니다.

In [19]:
import nltk
nltk.download('abc')
nltk.download('punkt')

[nltk_data] Downloading package abc to /aiffel/nltk_data...
[nltk_data]   Unzipping corpora/abc.zip.
[nltk_data] Downloading package punkt to /aiffel/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [22]:
from nltk.corpus import abc
corpus = abc.sents()

In [23]:
print(corpus[:3])

[['PM', 'denies', 'knowledge', 'of', 'AWB', 'kickbacks', 'The', 'Prime', 'Minister', 'has', 'denied', 'he', 'knew', 'AWB', 'was', 'paying', 'kickbacks', 'to', 'Iraq', 'despite', 'writing', 'to', 'the', 'wheat', 'exporter', 'asking', 'to', 'be', 'kept', 'fully', 'informed', 'on', 'Iraq', 'wheat', 'sales', '.'], ['Letters', 'from', 'John', 'Howard', 'and', 'Deputy', 'Prime', 'Minister', 'Mark', 'Vaile', 'to', 'AWB', 'have', 'been', 'released', 'by', 'the', 'Cole', 'inquiry', 'into', 'the', 'oil', 'for', 'food', 'program', '.'], ['In', 'one', 'of', 'the', 'letters', 'Mr', 'Howard', 'asks', 'AWB', 'managing', 'director', 'Andrew', 'Lindberg', 'to', 'remain', 'in', 'close', 'contact', 'with', 'the', 'Government', 'on', 'Iraq', 'wheat', 'sales', '.']]


In [24]:
print('코퍼스의 크기:', len(corpus))

코퍼스의 크기: 29059


이 코퍼스를 가지고 Word2Vec을 훈련시켜 보겠습니다.

In [25]:
from gensim.models import Word2Vec

model = Word2Vec(sentences = corpus, vector_size=100, window=5, min_count=5, workers=4, sg=0)

/opt/conda/lib/python3.7/site-packages/gensim/similarities/__init__.py:15: UserWarning: The gensim.similarities.levenshtein submodule is disabled, because the optional Levenshtein package <https://pypi.org/project/python-Levenshtein/> is unavailable. Install Levenhstein (e.g. `pip install python-Levenshtein`) to suppress this warning.
  warnings.warn(msg)


각 파라미터가 의미하는 바는 아래와 같습니다.  
* vector size = 학습 후 임베딩 벡터의 차원
* window = 컨텍스트 윈도우 크기
* min_count = 단어 최소 빈도수 제한 (빈도가 적은 단어들은 학습하지 않습니다.)
* workers = 학습을 위한 프로세스 수
* sg = 0은 CBoW, 1은 Skip-gram.  

Word2Vec는 입력한 단어에 대해서 가장 코사인 유사도가 높은 단어들을 출력하는 model.wv.most_similar를 지원합니다.

In [26]:
model_result = model.wv.most_similar("man")
print(model_result)

[('woman', 0.9233418107032776), ('skull', 0.911030113697052), ('Bang', 0.905648946762085), ('asteroid', 0.9052114486694336), ('third', 0.9020071625709534), ('baby', 0.8994219303131104), ('dog', 0.898607611656189), ('bought', 0.8975202441215515), ('rally', 0.8912495374679565), ('disc', 0.8889137506484985)]


일반적으로 더 많은 훈련 데이터를 사용하면 사용할수록 더 좋은 성능을 얻을 수 있습니다. 학습한 모델을 저장해두었다가 필요할 때 로드하면 좋습니다. 이번에는 모델을 저장하고 로드하는 방법을 배워보겠습니다

In [31]:
from gensim.models import KeyedVectors

model.wv.save_word2vec_format('~/aiffel/word_embedding/w2v') 
loaded_model = KeyedVectors.load_word2vec_format("~/aiffel/word_embedding/w2v")

In [32]:
model_result = loaded_model.most_similar("man")
print(model_result)

[('woman', 0.9233418107032776), ('skull', 0.911030113697052), ('Bang', 0.905648946762085), ('asteroid', 0.9052114486694336), ('third', 0.9020071625709534), ('baby', 0.8994219303131104), ('dog', 0.898607611656189), ('bought', 0.8975202441215515), ('rally', 0.8912495374679565), ('disc', 0.8889137506484985)]


### Word2Vec의 OOV 문제
Word2Vec은 Bag of words 학습 과정에서 언급되었던 문제인 사전에 없는 단어(Out Of Vocabuary) 문제를 그대로 가지고 있습니다. 다시 말해, 사전에 없는 단어에 대해서 Word2Vec은 임베딩 벡터값을 얻을 수 없습니다.   
"overacting"은 과장된 행동을 나타내는 단어입니다. 사실 자주 등장하는 단어는 아닌데요. overacting과 코사인 유사도가 가장 높은 단어를 출력해 보겠습니다.

In [33]:
loaded_model.most_similar('overacting')

KeyError: "Key 'overacting' not present"

"overacting"은 훈련 데이터에 없는 단어, 다시 말해 단어장에 존재하지 않는 단어므로 이 코드는 에러를 발생시킵니다. 에러 문구를 해석해보면 "단어 'overacting'은 단어장에 없습니다." 라는 의미입니다.  
이번에는 잘 등장하지 않아 훈련 데이터에 없는 단어라기보다는 오타가 발생했을 경우를 가정해보겠습니다. 테스트 데이터에 하필 단어 'memory'의 오타인 'memorry'라는 단어가 있었던 상황을 가정해보는 것입니다.

In [34]:
loaded_model.most_similar('memorry')

KeyError: "Key 'memorry' not present"

## FastText
페이스북에서 개발한 FastText는 Word2Vec 이후에 등장한 워드 임베딩 방법으로, 메커니즘 자체는 Word2Vec을 그대로 따르고 있지만, 문자 단위 n-gram(character-level n-gram) 표현을 학습한다는 점에서 다릅니다. Word2Vec은 단어를 더 이상 깨질 수 없는 단위로 구분하는 반면, FastText는 단어 내부의 내부 단어(subwords)들을 학습한다는 아이디어를 가지고 있습니다.  
FastText의 n-gram에서 n은 단어들이 얼마나 분리되는지 결정하는 하이퍼파라미터입니다. n을 3으로 잡은 트라이그램(tri-gram)의 경우, 단어 "partial"은 'par', 'art', 'rti', 'tia', 'ial'로 분리하고 이들을 벡터로 만듭니다. 더 정확히는 시작과 끝을 의미하는 <,>를 도입하여 <pa, art, rti, tia, ial, al>라는 6개의 내부 단어(subword) 토큰을 벡터로 만듭니다. 여기에 추가적으로 하나를 더 벡터화하는데, 기존 단어에 <, 와 >를 붙인 토큰 <partial>입니다.  
즉 n = 3인 경우, FastText는 단어 partial에 대해 임베딩되는 n-gram 토큰들은 다음과 같습니다.

n = 3인 경우  
<pa, art, rti, tia, ial, al>, <partial>

실제 사용할 때는 n의 최솟값과 최댓값으로 범위를 설정할 수 있는데, gensim 패키지에서는 기본값으로 각각 3과 6으로 설정되어 있습니다. 다시 말해 최솟값 = 3, 최댓값 = 6인 경우라면, 단어 "partial"에 대해서 FastText는 아래 내부 단어들을 벡터화합니다.

n = 3 ~ 6인 경우  
<pa, art, rti, ita, ial, al>, <par, arti, rtia, tial, ial>, <part, ...중략... , <partial>

여기서 내부 단어들을 벡터화한다는 의미는 저 단어들 각각에 대해서 Word2Vec을 수행한다는 의미입니다. 최종적으로 이렇게 벡터화된 n-gram 벡터들의 총합을 해당 단어의 벡터로 취합니다.

각 원소는 벡터임을 가정함  
partial = <pa + art + rti + ita + ial + al> + <par + arti + rtia + tial + ial> + <part + ...중략...  + <partial>

### FastText의 학습 방법
FastText의 학습 방식은 Word2Vec와 크게 다르지 않습니다. FastText도 Word2Vec과 마찬가지로 네거티브 샘플링을 사용하여 학습합니다. "(중심 단어, 주변 단어)"의 쌍을 가지고 이 쌍이 포지티브인지 네거티브인지 예측을 진행하는 것이죠. 다만, Word2Vec과 다른 점은 학습 과정에서 중심 단어에 속한 문자 단위 n-gram 단어 벡터들을 모두 업데이트한다는 점입니다.

### OOV와 오타에 대한 대응
FastText는 Word2Vec과 달리 OOV와 오타에 강건하다)robust)는 특징이 있습니다. 이는 단어장에 없는 단어라도, 해당 단어의 n-gram이 다른 단어에 존재하면 이로부터 벡터값을 얻는다는 원리에 기인합니다.

In [35]:
from gensim.models import FastText
fasttext_model = FastText(corpus, window=5, min_count=5, workers=4, sg=1)

이제 Word2Vec에서 에러가 발생했던 단어들을 FastText 모델에 입력해 보겠습니다.

In [36]:
fasttext_model.wv.most_similar('overacting')

[('resolving', 0.940370500087738),
 ('fluctuating', 0.9384295344352722),
 ('emptying', 0.934513509273529),
 ('malting', 0.9311069250106812),
 ('mounting', 0.9303527474403381),
 ('declining', 0.9298065900802612),
 ('shooting', 0.9295960664749146),
 ('extracting', 0.9294117093086243),
 ('overwhelming', 0.9292879700660706),
 ('debilitating', 0.9290804862976074)]

'overacting'이 단어장에 없던 단어임에도 정상적으로 임베딩 벡터값이 계산되어 유사 단어 10개를 출력하는 것을 볼 수 있습니다.

In [37]:
fasttext_model.wv.most_similar('memoryy')

[('memory', 0.9468633532524109),
 ('musical', 0.8672632575035095),
 ('mechanism', 0.860868513584137),
 ('mechanisms', 0.8601607084274292),
 ('basic', 0.8556936979293823),
 ('mechanical', 0.8514925241470337),
 ('imagine', 0.8477826118469238),
 ('technical', 0.8415030837059021),
 ('intelligence', 0.8390647172927856),
 ('specific', 0.8328201174736023)]

### 한국어에서의 FastText
한국어도 당연히 FastText 방식으로 학습시킬 수 있습니다. 영어의 경우 문자, 즉 알파벳 단위가 n-gram이었다면 한국어의 경우에는 음절 단위라고 볼 수 있습니다.

1. 음절 단위 FastText  
n = 3일때, 단어 '텐서플로우'의 트라이그램 벡터들은 다음과 같이 구성됩니다.  
* <텐서, 텐서플, 서플로, 플로우, 로우>, <텐서플로우>  
한국어에서 FastText가 빛을 발하는 것은 사실 음절 단위라기보다는 자소 단위인 경우입니다. 한국어의 자소를 각각의 문자로 간주한 경우에 FastText는 꽤 잘 동작한다고 알려져 있습니다.

2. 자소 단위 FastText  
단어에 대해서 초성, 중성, 종성을 분리한다고 하고, 종성이 존재하지 않는 경우에는 _라는 토큰을 대신 사용한다면  n = 3일 때, 단어 '텐서플로우'는 다음과 같이 트라이그램 벡터들로 분리할 수 있습니다. <ㅌㅔ,ㅌㅔㄴ,ㅔㄴㅅ,ㄴㅅㅓ,ㅅㅓ_, ...중략... >

## GloVe
글로브(Global Vectors for Word Representation, GloVe)는 2014년에 미국 스탠포드 대학에서 개발한 워드 임베딩 방법론입니다. 워드 임베딩의 두 가지 접근 방법인 카운트 기반과 예측 기반 두 가지 방법을 모두 사용했다는 것이 특징입니다.  
카운트 기반 방법이라는 것은 어떤 의미일까요? 앞서 단어의 빈도를 수치화한 방법인 DTM을 배웠었죠. DTM의 경우에는 단어 간 유사도를 반영할 수 없을 뿐만 아니라, 대부분의 값이 0인 희소 표현이라는 특징이 있었습니다. DTM을 차원 축소하여 밀집 표현(dense representation)으로 임베딩 하는 방법이 LSA(Latent Semantic Analysis)입니다.

LSA를 요약하면 DTM에 특잇값 분해를 사용하여 잠재된 의미를 이끌어내는 방법론입니다. 그 결과의 행벡터를 사용해서 임베딩 벡터를 얻을 수도 있습니다. LSA는 단어를 카운트해서 만든 DTM을 입력으로 하므로 카운트 기반의 임베딩 방법이라고 볼 수 있는데, 이 방법은 몇 가지 한계가 있었습니다.

(1) 차원 축소의 특성으로 인해 새로운 단어가 추가되면 다시 DTM을 만들어 새로 차원 축소를 해야 한다.  
(2) 단어 벡터간 유사도를 계산하는 측면에서 Word2Vec보다 성능이 떨어진다.

반면, LSA와 대조되는 방법으로 예측 기반의 방법은 Word2Vec과 같은 방법을 말합니다. Word2Vec은 인공 신경망이 예측한 값으로부터 실제 레이블과의 오차를 구하고, 손실 함수를 통해서 인공 신경망을 학습하는 방식입니다. GloVe 연구진은 Word2Vec의 경우에는 LSA보다 단어 벡터 간 유사도를 구하는 능력은 뛰어나지만, LSA처럼 코퍼스의 전체적인 통계 정보를 활용하지는 못한다는 점을 한계로 지적했습니다. 그리고는 카운트 기반과 예측 기반을 모두 사용하여 Word2Vec보다 더 나은 임베딩 방법을 제안하였는데, GloVe이 그것입니다. 하지만 GloVe가 Word2Vec보다 반드시 뛰어나다고 장담하기는 어렵고, Word2Vec에 거의 준하는 성능을 보여준다고 평가되고 있습니다.

### 윈도우 기반 동시 등장 행렬(Window based Co-occurrence Matrix)
GloVe를 이해하기 위해서는 윈도우 기반 동시 등장 행렬의 정의에 대해서 이해할 필요가 있습니다.  
윈도우 기반 동시 등장 행렬은 행과 열을 전체 단어장(vocabulary)의 단어들로 구성하고, 어떤 i 단어의 윈도우 크기(window Size) 내에서 k 단어가 등장한 횟수를 i행 k열에 기재한 행렬입니다. 이러한 동시 등장 행렬은 전치(transpose)해도 동일한 행렬이 된다는 특징을 가지고 있습니다.

### 동시 등장 확률(Co-occurrence Probability)
동시 등장 확률 P (k|i)P(k∣i)는 동시 등장 행렬로부터 특정 단어 i의 전체 등장 횟수를 카운트하고, 특정 단어 i가 등장했을 때 어떤 단어 k가 등장한 횟수를 카운트하여 계산한 조건부 확률입니다. 이때 i를 중심 단어(center word), k를 주변 단어(context word)라고 합니다.

### GloVe의 손실 함수
GloVe는 동시 등장 행렬로부터 계산된 동시 등장 확률을 이용해 손실 함수를 설계합니다. 동시 등장 행렬을 사용하고 있으니 코퍼스의 전체적인 통계 정보를 활용하는 '카운트 기반'의 방법론이면서, 손실 함수를 통해 모델을 학습시키므로 '예측 기반'의 방법론이라고 할 수 있는 것이죠.  
GloVe의 아이디어를 요약하면 다음과 같습니다.  
중심 단어 벡터와 주변 단어 벡터의 내적이 전체 코퍼스에서의 동시 등장 빈도의 로그값이 되도록 만드는 것  
즉, 전체 코퍼스에서의 동시 등장 빈도의 로그값과 중심 단어 벡터와 주변 단어 벡터의 내적값의 차이가 최소화되도록 두 벡터의 값을 학습하는 것입니다.

### GloVe 실습

NLTK에서 제공하는 영화 리뷰 데이터를 다운로드해 corpus 변수에 저장합니다.

In [39]:
import nltk
nltk.download('movie_reviews')
nltk.download('punkt')

[nltk_data] Downloading package movie_reviews to /aiffel/nltk_data...
[nltk_data]   Unzipping corpora/movie_reviews.zip.
[nltk_data] Downloading package punkt to /aiffel/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [40]:
from nltk.corpus import movie_reviews
corpus=movie_reviews.sents()

In [41]:
from glove import Corpus, Glove

# 훈련 데이터로부터 GloVe에서 사용할 동시 등장 행렬 생성
emb = Corpus() 
emb.fit(corpus, window=5)

# 벡터의 차원은 100, 학습에 이용할 쓰레드의 개수는 4로 설정, 에포크는 20.
glove = Glove(no_components=100, learning_rate=0.05)
glove.fit(emb.matrix, epochs=20, no_threads=4, verbose=True)
glove.add_dictionary(emb.dictionary)

Performing 20 training epochs with 4 threads
Epoch 0
Epoch 1
Epoch 2
Epoch 3
Epoch 4
Epoch 5
Epoch 6
Epoch 7
Epoch 8
Epoch 9
Epoch 10
Epoch 11
Epoch 12
Epoch 13
Epoch 14
Epoch 15
Epoch 16
Epoch 17
Epoch 18
Epoch 19


man, fiction과 유사한 단어들을 확인하려면 아래와 같은 코드를 실행시키면 됩니다.

In [42]:
model_result1 = glove.most_similar("man")
model_result2 = glove.most_similar("fiction")

print("model_result1", model_result1)
print("model_result2", model_result2)

model_result1 [('woman', 0.9537157869200451), ('young', 0.8926166938265798), ('girl', 0.8920057462920079), ('boy', 0.8875882582848577)]
model_result2 [('science', 0.9835003571234929), ('pulp', 0.9643030076374081), ('rocky', 0.7309243308420116), ('kong', 0.707728416375735)]
